# Sequential removal of links and resiliency testing

- Disruption of a network by removal of links, based on:
    + Sum of betweeness centrality of from and to nodes
    + Link length
    + Volume of commodity flow
- Calculation of performance in terms of cost and unmet demand by re-running disrupted network on FOT
- Plot link removal along x-axis and performance on y-axis, comparing networks of differing evenness. Dynamic report generated in an Rmarkdown automatically from this Notebook.

**Assumptions**

- Working in a Python 3.x environment for this notebook
    + Refer to the README in this repository for instructions on setup of all dependencies with `conda`
- Python 2.7 installed as part of ArcGIS
- 64 bit background geoprocessing enabled
- Access to ArcGIS license server if necessary 

*Reference*

- [NetworkX Documentation](https://networkx.github.io/documentation/stable/tutorial.html)

In [1]:
import pandas as pd
import geopandas as gpd
import sqlalchemy 
import networkx as nx
import os
import matplotlib.pyplot as plt
import pickle

import resiliency_disruptions

# Uses Quick Start 7 as an example. Modify `scen_name` and `scen_path` for your scenario.
scen_name = 'qs7_rmp_proc_dest_multi_inputs\\Default'

scen_path = os.path.join("C:\\FTOT\\scenarios\\quick_start\\", scen_name)

shp_path = os.path.join(scen_path, 'temp_networkx_shp_files')

picklename = os.path.join(scen_path, 'BetweenessG.pickle')

if not os.path.exists(shp_path):
    print('Please modify the FTOT code using the `ftot_networkx.py` and `ftot_routing.py` scripts in this repository and run the scenario again.')

In [2]:
# Read in prepared betweeness centrality and road network graph data. 
# If these don't exist, the following steps will create them
if os.path.exists(picklename):
    file = open(picklename, 'rb')
    betweenness_dict_road = pickle.load(file)
    G_road = pickle.load(file)

In [3]:
# Start by using betweeness centrality calculation using networkX
if not os.path.exists(picklename):
    G_road = nx.read_shp(os.path.join(shp_path, 'road.shp'), simplify=True)
    G_road = nx.convert_node_labels_to_integers(G_road, first_label=0, ordering='default', label_attribute="xy_coord_label")
    

In [4]:
# Run betweenness centrality on the NetworkX graph
# Note: This step might take 20+ minutes
# Run if pickle not available
if not os.path.exists(picklename):
    print('Running Betweeness Centrality calculations. This might take more than 20 minutes.')
    betweenness_dict_road = nx.betweenness_centrality(G_road, normalized=False, weight='MILES')
    print('Completed Betweeness Centrality calculations.')


In [5]:
# Save with pickle
# On load, need to know that there are two objects in this pickle, the betweeness centrality dict and the network G
if not os.path.exists(picklename):
    with open(picklename, 'wb') as handle:
        pickle.dump(betweenness_dict_road, handle)
        pickle.dump(G_road, handle)

## Join Betweeness Centrality calculations to edges 

- Sum BC for each node of a link
- Create data frame for repeated link removal

In [6]:
# Read in FTOT data
print(scen_path)
db_name = 'main.db'

db_path = 'sqlite:///' + os.path.join(scen_path, db_name)

engine = sqlalchemy.create_engine(db_path)

table_name = 'networkx_edges'
nx_edges = pd.read_sql_table(table_name, engine)

table_name = 'networkx_nodes'
nx_nodes = pd.read_sql_table(table_name, engine)

table_name = 'optimal_variables'
optimal_vars = pd.read_sql_table(table_name, engine)

C:\FTOT\scenarios\quick_start\qs7_rmp_proc_dest_multi_inputs\Default


In [7]:
G_road_orig_label = nx.read_shp(os.path.join(shp_path, 'road.shp'), simplify=True)

C:\Users\DANIEL~1.FLY\AppData\Local\Temp/ipykernel_16580/1918546944.py:1: DeprecationWarning: read_shp is deprecated and will be removed in 3.0.See https://networkx.org/documentation/latest/auto_examples/index.html#geospatial.
  G_road_orig_label = nx.read_shp(os.path.join(shp_path, 'road.shp'), simplify=True)


In [8]:
road_orig_label_nodes = list(G_road_orig_label.nodes) # these values are the shape_x and shape_y values in `networkx_nodes`. 
# Use that to get node_id from networkx_edges in the database,
# Then use those id values to get edges info
# Then line up the new integer labels with this list of ids to get betweeness centrality for each node

In [9]:
# Make the betweeness_centrality values as the framework to join in shape_x, shape_y, and node_id
bc_df_road = pd.DataFrame.from_dict(betweenness_dict_road, orient = 'index')
bc_df_road = bc_df_road.rename(columns = {0: 'BC'})

In [10]:
node_shape_df_road = pd.DataFrame(road_orig_label_nodes)

bc_shape_df_road = pd.concat([bc_df_road, node_shape_df_road], axis = 1)
bc_shape_df_road = bc_shape_df_road.rename(columns = {0: 'shape_x', 1: 'shape_y'})

# Now add node_id from networkx_nodes, using pandas merge with left join.
# Use both shape_x and shape_y to identify the nodes correctly
# Union of both prod and crude now

bc_node_df = pd.merge(bc_shape_df_road, nx_nodes, on = ['shape_x', 'shape_y'], how = 'left')

In [11]:
# Now use this data frame to populate a data frame of edges. 
# We will want the following from networkx_edges:
# edge_id, from_node_id, to_node_id, mode_source, miles, mode_source_oid, 
# Then using the node_id column in the new bc_node_df, add these:
# from_node_BC, to_node_BC
# and sum those for sum_node_BC
merge_from = pd.merge(nx_edges, bc_node_df[['BC','node_id']],
                      left_on = 'from_node_id',
                      right_on = 'node_id',
                      how = 'left')
merge_from = merge_from.rename(columns = {'BC': 'from_node_BC'})

merge_to = pd.merge(merge_from, bc_node_df[['BC','node_id',]],
                    left_on = 'to_node_id',
                    right_on = 'node_id',
                    how = 'left')
merge_to = merge_to.rename(columns = {'BC': 'to_node_BC'})

# Sum the BC values

merge_to['sum_BC'] = merge_to.filter(like = "node_BC").sum(axis = 1)

# Then from optimal_variables, get variable_name, nc_edge_id, mode, mode_oid, miles,
# variable_value, converted_capacity, and converted_volume

use_opt_vars = ['variable_type',
               'var_id',
               'variable_value',
                'variable_name',
                'nx_edge_id',
                'mode_oid',
                'converted_capacity',
                'converted_volume'
               ]

merge_opt = pd.merge(merge_to, optimal_vars[use_opt_vars],
                    left_on = 'edge_id',
                    right_on = 'nx_edge_id',
                    how = 'left')

In [12]:
merge_opt.head(3)

,edge_id,from_node_id,to_node_id,artificial,mode_source,mode_source_oid,miles,route_cost_scaling,capacity,volume,...,node_id_y,sum_BC,variable_type,var_id,variable_value,variable_name,nx_edge_id,mode_oid,converted_capacity,converted_volume
0,1,0,2222,2,rail,206,0.013221,1.0,NaN,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0,42181,2,water,5081,0.351856,1.0,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,2223,2,rail,207,0.064799,1.0,NaN,0.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Create ranked lists of edges to remove.
# First, keep only edges in the optimal solution.
# Then rank by sum_BC. Then just keep the columns we need, and reset the index.
use_cols = ['edge_id', 'from_node_id', 'to_node_id', 'miles', 'capacity', 'volume', 'sum_BC',
           'variable_type', 'variable_value', 'variable_name', 'nx_edge_id', 'mode_oid', 'converted_capacity', 'converted_volume']

edges_remove = merge_opt[merge_opt['variable_value'] > 0].sort_values(by = 'sum_BC', ascending = False).filter(items = use_cols).reset_index()

edges_remove.to_csv(os.path.join(scen_path, 'Edges_to_Remove.csv'),
                   index = False)

edges_remove.head(3)

,index,edge_id,from_node_id,to_node_id,miles,capacity,volume,sum_BC,variable_type,variable_value,variable_name,nx_edge_id,mode_oid,converted_capacity,converted_volume
0,52651,52652,23240,22927,2.391027,45396.405255,44631.0,12291580.0,Edge,90.718474,Edge_157928,52652.0,12925.0,1.089514e+06,1071144.0
1,51780,51781,22901,23033,3.527136,52814.692258,52543.0,10650212.0,Edge,90.718474,Edge_155315,51781.0,12949.0,1.267553e+06,1261032.0
2,51839,51840,22927,22928,1.124715,50570.979389,46904.0,10300608.0,Edge,90.718474,Edge_155492,51840.0,12525.0,1.213704e+06,1125696.0


## Create Scenarios, Disrupt, Run FTOT

Create disrupted network by copying everyhing in `scen_path` to a new directory

Then overwrites the `networkx_edges` tables in that main.db, with the disrupted versions.

##### Assuptions:

  1. ArcGIS with 64-bit geoprocessing is installed
  2. The FTOT version being used has been modified according to the `README` in this directory


In [14]:
disrupt_type = 'BC' # Can disrupt basaed on betweeness centrality or volume, 'V'
disrupt_steps = 25  # This is the number of steps to use. Recommend at least 25.

resiliency_disruptions.make_disruption_scenarios(disrupt_type, disrupt_steps, scen_path)

Prepared 25 scenarios based on Default


In [15]:
resiliency_disruptions.disrupt_network(disrupt_type, disrupt_steps, scen_path, edges_remove)

Disrupted 25 scenarios


In [16]:
PYTHON = r"C:\FTOT\python3_env\python.exe"
repo_location = %pwd
repo_location = os.path.split(repo_location)[0] 
FTOT = r"C:\FTOT\program\ftot.py" # Optionally: os.path.join(repo_location, 'program', 'ftot.py')
print(FTOT)

C:\FTOT\program\ftot.py


In [17]:
# Begin running O steps of FTOT on the disupted scenarios
# This may take several hours, depending on size of the network and number of steps

results = resiliency_disruptions.run_o_steps(disrupt_type, disrupt_steps, scen_path, PYTHON, FTOT)

Running o1 for disrupt01
Running o2 for disrupt01
Running p for disrupt01
Running d for disrupt01
Preparing to search over o2_log_2022_03_15_19-58-38.log
  disrupt_step unmet_demand unmet_cost nedge total_cost
0           01            0          0   263      3,643
Running o1 for disrupt02
Running o2 for disrupt02
Running p for disrupt02
Running d for disrupt02
Preparing to search over o2_log_2022_03_15_20-02-33.log
  disrupt_step unmet_demand unmet_cost nedge total_cost
0           02            0          0   263      3,643
Running o1 for disrupt03
Running o2 for disrupt03
Running p for disrupt03
Running d for disrupt03
Preparing to search over o2_log_2022_03_15_20-04-33.log
  disrupt_step unmet_demand unmet_cost nedge total_cost
0           03            0          0   263      3,643
Running o1 for disrupt04
Running o2 for disrupt04
Running p for disrupt04
Running d for disrupt04
Preparing to search over o2_log_2022_03_15_20-06-32.log
  disrupt_step unmet_demand unmet_cost nedge tot

In [18]:
results

,index,disrupt_step,unmet_demand,unmet_cost,nedge,total_cost
0,0,01,0,0,263,"3,643"
1,0,02,0,0,263,"3,643"
2,0,03,0,0,263,"3,643"
3,0,04,0,0,263,"3,643"
4,0,05,0,0,263,"3,643"
5,0,06,0,0,263,"3,643"
6,0,07,0,0,214,"3,657"
7,0,08,0,0,214,"3,657"
8,0,09,0,0,214,"3,657"
9,0,10,0,0,214,"3,657"


#### Optional: Repeat with volume-based disruptions

Creates a separate directory tree for the volume-based disruptions, and carries out the disruption steps on that set.

Set the variable `DO_VOLUME` to `True` to run the following steps

In [19]:
DO_VOLUME = False

if DO_VOLUME:

    disrupt_type = 'V'
    disrupt_steps = 50

    resiliency_disruptions.make_disruption_scenarios(disrupt_type, disrupt_steps, scen_path)
    resiliency_disruptions.disrupt_network(disrupt_type, disrupt_steps, scen_path, edges_remove)
    results = resiliency_disruptions.run_o_steps(disrupt_type, disrupt_steps, scen_path, PYTHON, FTOT)
    results

## Generate disruption result report

Run `compile_report.py`, which generates the `Disruption_Results.html` report.


In [20]:
import compile_report

compile_report.main()